In [2]:
import open3d as o3d
from o3d_tools.visualize import PointCloudProject
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import torch
from torch_geometric.data import Data
import torch_geometric.transforms as T
from torch_geometric.data import Dataset, DataLoader
import torch_cluster
import torch.nn.functional as F
from torch_geometric.nn import DynamicEdgeConv, global_max_pool
import torch.optim as optim

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
project1 = PointCloudProject(project='Project1') ; project2 = PointCloudProject(project='Project2')
project3 = PointCloudProject(project='Project3') ; project4 = PointCloudProject(project='Project4')

### Preprocess data

In [4]:
def read_and_preprocess_data():

    objs1 = project1.objects_df

    print(f"Types of objects in Project 1: {list(objs1.keys())}")
    pipes1 = objs1["Pipe"]
    scbs1 = objs1["Structural_ColumnBeam"]
    sibs1 = objs1["Structural_IBeam"]
    hvducts = objs1["HVAC_Duct"]

    objs2 = project2.objects_df

    print(f"Types of objects in Project 2: {list(objs2.keys())}")
    pipes2 = objs2["Pipe"]
    scbs2 = objs2["Structural_ColumnBeam"]
    sibs2 = objs2["Structural_IBeam"]

    objs3 = project3.objects_df

    print(f"Types of objects in Project 3: {list(objs3.keys())}")
    scbs3 = objs3["Structural_ColumnBeam"]

    objs4 = project4.objects_df

    print(f"Types of objects in Project 4: {list(objs4.keys())}")
    sibs4 = objs4["Structural_IBeam"]


    boxes1 = pd.concat([scbs1, hvducts, pipes1, sibs1])
    boxes2 = pd.concat([scbs2, pipes2, sibs2])
    boxes3 = scbs3
    boxes4 = sibs4

    boxes = [boxes1, boxes2, boxes3, boxes4]

    for i in range(len(boxes)):
        boxes[i].rename(columns={" Label": "label", " BB.Min.X " : "min_x", " BB.Min.Y " : "min_y", " BB.Min.Z " : "min_z",
                             " BB.Max.X " : "max_x", " BB.Max.Y " : "max_y", " BB.Max.Z" : "max_z"}, inplace=True)

    box_data = pd.concat(boxes)

    ids_to_points = {}

    projects = [project1, project2, project3, project4]

    for proj_id, box in enumerate(boxes):

        for i, ID in enumerate(box["ID"]):

            proj = projects[proj_id]

            # .pcd.crop() method expects this bounding box object, create it from the raw min max values
            bb = o3d.geometry.AxisAlignedBoundingBox((box[["min_x", "min_y", "min_z"]].iloc[i]).to_numpy(), 
                                        (box[["max_x", "max_y", "max_z"]].iloc[i]).to_numpy())

            points_of_id = proj.pcd.crop(bb)

            # convert the pointcloud object into a numpy array
            # join coordinates and color data into a single 6d nparray for each point

            points_coords = np.asarray(points_of_id.points)

            tmp = np.asarray(points_of_id.colors)[:,0]

            points_colors = (np.asarray(points_of_id.colors)[:,0]).reshape(-1,1)

            points_arr = np.concatenate((points_coords, points_colors), axis = 1)

            ids_to_points[ID] = points_arr

    def sample_point_cloud(points, num_points):

        if points.shape[0] > num_points:

            # Downsample if the point cloud has more points than needed
            idx = np.random.choice(points.shape[0], num_points, replace=False)
            return points[idx, :]

        elif points.shape[0] < num_points:
            # Upsample by repeating random points if the point cloud has fewer points than needed
            idx = np.random.choice(points.shape[0], num_points - points.shape[0], replace=True)
            return np.concatenate([points, points[idx, :]], axis=0)

        else:
            # Return the point cloud as is if it already has the correct number of points
            return points

    fixed_num_points = 2048  # The fixed number of points for all point clouds

    point_clouds_resampled = {}

    for obj_id in ids_to_points.keys():

        point_cloud = ids_to_points[obj_id]
        resampled_points = sample_point_cloud(point_cloud, fixed_num_points)
        point_clouds_resampled[obj_id] = resampled_points
        
    def extract_bounding_box_features(min_x, max_x, min_y, max_y, min_z, max_z):
    
        centroid = [(min_x + max_x) / 2, (min_y + max_y) / 2, (min_z + max_z) / 2]
        size = [max_x - min_x, max_y - min_y, max_z - min_z]
        volume = size[0] * size[1] * size[2]
        
        return np.array(centroid + size + [volume])
    
    data = []
    for i in range(box_data.shape[0]):

        box = box_data.iloc[i]
        features = extract_bounding_box_features(box["min_x"], box["max_x"], box["min_y"],
                                                 box["max_y"], box["min_z"], box["max_z"])

        data.append(features)

    feature_df = pd.DataFrame(data, columns=["cx", "cy", "cz", "sx", "sy", "sz", "vol"])
    
    box_data = box_data.reset_index(drop=True)
    
    new_box_data = pd.concat([box_data, feature_df], axis=1)
    
    return point_clouds_resampled, new_box_data

In [5]:
points_dict, box_data = read_and_preprocess_data()

Types of objects in Project 1: ['Structural_ColumnBeam', 'HVAC_Duct', 'Pipe', 'Structural_IBeam']
Types of objects in Project 2: ['Structural_ColumnBeam', 'Pipe', 'Structural_IBeam']
Types of objects in Project 3: ['Structural_ColumnBeam']
Types of objects in Project 4: ['Structural_IBeam']


### Split into train val test

In [6]:
y = box_data["label"]
X = box_data[["ID", "min_x", "min_y", "min_z", "max_x", "max_y", "max_z", "cx", "cy", "cz", "sx", "sy", "sz", "vol"]]

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

### Encode labels as integers

In [7]:
label_mapping = {
    'Structural_ColumnBeam': 0,
    'HVAC_Duct': 1,
    'Pipe': 2,
    'Structural_IBeam': 3
}

labels_enc = []
for row in box_data["label"]:

    labels_enc.append(label_mapping[row])

In [8]:
box_data["label_enc"] = labels_enc

### Create data object and graph features

In [9]:
def create_point_cloud_data(points, bounding_box_features, labels):
    
    # Points is of shape [2048, 3] (x, y, z coordinates)
    # Grayscale intensity is of shape [2048, 1] 
    # Labels are the classification targets for each bounding box

    x = torch.tensor(points, dtype=torch.float)  # Point cloud coordinates
    #print(x)
    #print(x.shape)

    # Create an edge index using k-nearest neighbors (or some method to define neighbors)
    edge_index = T.KNNGraph(k=16)(Data(pos=x)).edge_index  # Edge index for the graph

    bounding_box_features = torch.tensor(bounding_box_features, dtype=torch.float)

    # Create the Data object
    data = Data(x=x, edge_index=edge_index, y=torch.tensor(labels, dtype=torch.long), bbox_features=bounding_box_features)

    return data

In [10]:
graph_dict = {}
features_list = ["cx", "cy", "cz", "sx", "sy", "sz", "vol"]

for obj_id in points_dict.keys():

    curr_label = ((box_data[box_data["ID"] == obj_id].iloc[:,15]).to_numpy()).reshape(1)
    curr_features = ((box_data[box_data["ID"] == obj_id].loc[:,features_list]).to_numpy()).reshape(7)
    
    graph_dict[obj_id] = create_point_cloud_data(points_dict[obj_id], curr_features, curr_label)


In [16]:
print(graph_dict)

{'d442ca7c-770d-4bc2-a565-81b971e1b3b3': Data(x=[2048, 4], edge_index=[2, 32768], y=[1], bbox_features=[7]), 'de9a038a-e106-48fa-ac76-3f83dfa1503a': Data(x=[2048, 4], edge_index=[2, 32768], y=[1], bbox_features=[7]), 'b351dae6-0cf2-45f2-bbfd-ab9e607bce0d': Data(x=[2048, 4], edge_index=[2, 32768], y=[1], bbox_features=[7]), '62e75328-ddda-48d4-87c4-210afb055eb5': Data(x=[2048, 4], edge_index=[2, 32768], y=[1], bbox_features=[7]), '64ca25ff-ebc8-4e83-9585-756d6b711a3c': Data(x=[2048, 4], edge_index=[2, 32768], y=[1], bbox_features=[7]), '7296381a-9ab0-405c-85f7-89ea5e1aa903': Data(x=[2048, 4], edge_index=[2, 32768], y=[1], bbox_features=[7]), 'f2990cfa-4762-42ae-9930-5b097771c5c7': Data(x=[2048, 4], edge_index=[2, 32768], y=[1], bbox_features=[7]), '537952e4-2ef2-45c6-b164-e975ccfd6c87': Data(x=[2048, 4], edge_index=[2, 32768], y=[1], bbox_features=[7]), '52fbd341-ae6d-4c61-be37-32e05953cf86': Data(x=[2048, 4], edge_index=[2, 32768], y=[1], bbox_features=[7]), '8f378c1e-f586-4da8-8440-88

In [11]:
train_data_objs = {}
val_data_objs = {}
test_data_objs = {}

for key in X_train["ID"]:

    train_data_objs[key] = graph_dict[key]

for key in X_val["ID"]:

    val_data_objs[key] = graph_dict[key]

for key in X_test["ID"]:

    test_data_objs[key] = graph_dict[key]

In [12]:
train_data_arr = []

for key in train_data_objs.keys():

    train_data_arr.append(train_data_objs[key])

In [13]:
class PointCloudWithBoundingBoxModel(torch.nn.Module):
    def __init__(self, k=16, num_classes=4, bbox_input_size=7):
        super(PointCloudWithBoundingBoxModel, self).__init__()
        self.k = k

        # GNN for Point Cloud Data
        self.conv1 = DynamicEdgeConv(torch.nn.Sequential(
            torch.nn.Linear(4 * 2, 32),  # 4 features (x, y, z, intensity) * 2
            torch.nn.ReLU(),
            torch.nn.Linear(32, 32)
        ), k=self.k)

        self.conv2 = DynamicEdgeConv(torch.nn.Sequential(
            torch.nn.Linear(32 * 2, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 128)
        ), k=self.k)

        self.conv3 = DynamicEdgeConv(torch.nn.Sequential(
            torch.nn.Linear(128 * 2, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 256)
        ), k=self.k)

        # Fully Connected Layers for Bounding Box Features
        self.fc_bbox = torch.nn.Sequential(
            torch.nn.Linear(bbox_input_size, 32),
            torch.nn.ReLU(),
            torch.nn.Linear(32, 128)
        )

        # Fully Connected Layers for the combined output
        self.fc_final = torch.nn.Sequential(
            torch.nn.Linear(256 + 128, 512),  # 256 from point cloud GNN, 128 from bbox features
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3),
            torch.nn.Linear(512, num_classes)
        )

    def forward(self, data):
        x, batch = data.x, data.batch
        x = self.conv1(x, batch)
        x = self.conv2(x, batch)
        x = self.conv3(x, batch)
        x = global_max_pool(x, batch)

        bbox_features = data.bbox_features
        bbox_out = self.fc_bbox(bbox_features)

        x = torch.cat([x, bbox_out], dim=1)
        x = self.fc_final(x)

        return x  # Remove log_softmax; use CrossEntropyLoss directly


In [14]:
batch_size = 32  # Adjust based on your resources
data_loader = DataLoader(train_data_arr, batch_size=batch_size, shuffle=True)

/home/ubuntu/.local/lib/python3.9/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [20]:
model = PointCloudWithBoundingBoxModel()

In [15]:
for data in data_loader:
    print(data.bbox_features.shape)

torch.Size([224])
torch.Size([224])
torch.Size([224])
torch.Size([224])
torch.Size([224])
torch.Size([224])
torch.Size([224])
torch.Size([224])
torch.Size([98])


In [ ]:
for data in data_loader:
    out = model(data)